In [166]:
import numpy as np


In [169]:
def qr(A):
    rand=np.random.randn(*np.shape(A))
    Q,_=np.linalg.qr(A)
    i=0
    while True:
        i=i+1
        Z=A.dot(Q)
        Q_new,R_new=np.linalg.qr(Z)
        if np.linalg.norm(Q_new-Q)<0.000001:
            break
        if i==10000:
            break
        Q=Q_new
    return R_new, Q
        
        
        
    

In [237]:
def cs(alpha,beta):
    asq=np.square(alpha)
    bsq=np.square(beta)
    c=alpha/np.sqrt(asq+bsq)
    s=beta/np.sqrt(asq+bsq)
    return c,s

In [238]:
def lg(n,k,c,s):
    a=np.identity(n)
    a[k,k]=c
    a[k,k+1]=s
    a[k+1,k]=-s
    a[k+1,k+1]=c
    return a

In [239]:
def rg(n,k,c,s):
    a=np.identity(n)
    a[k,k]=c
    a[k,k+1]=-s
    a[k+1,k]=s
    a[k+1,k+1]=c
    return a

In [303]:
def R(a):
    n,m=np.shape(a)
    G=[]
    for i in range(0,n-1):
        alpha=a[i,i]
        beta=a[i+1,i]
        g=lg(n,i,*cs(alpha,beta))
        G.append(g.T)
        a=np.matmul(g,a)
    return a,G,n

def Q(G,n):
    q=np.identity(n)
    for i in G:
        q=np.matmul(q,i)
    return q
def qrdecomp(a):    
    for i in range(0,13):
        r,g,n=R(a)
        q=Q(g,n)
        a=np.matmul(r,q)
        print(np.around(a))
    return a
        
        
    
    

In [304]:
###########House Holder Transform##########################
import numpy as np

In [305]:
def hht(arr,index,column):
    matrix=np.array(arr)
    n,m=np.shape(matrix)
    #print("n={}".format(n))
    j=index-1
    column=column-1
    alpha=np.sqrt(np.sum(matrix[j:,column]*matrix[j:,column]))*np.sign(matrix[j,column])*np.array([-1])
    r=np.sqrt(0.5*alpha*alpha-0.5*matrix[j,column]*alpha)
    if j!=0 and j!=n-2:
        w0=np.zeros(j)
        w1=(matrix[j,column]-alpha)/(2*r)
        wj=np.array([matrix[i,column]*0.5/r for i in range(j+1,n)])
        w=np.array(np.concatenate((w0,w1,np.squeeze(wj))))
    elif j==n-2:
        w0=np.zeros(j)
        w1=(matrix[j,column]-alpha)/(2*r)
        wj=np.array([matrix[i,column]*0.5/r for i in range(j+1,n)])
        w=np.array(np.concatenate((w0,w1,np.reshape(wj,(1)))))
    else:
        w1=(matrix[j,column]-alpha)/(2*r)
        wj=np.array([matrix[i,column]*0.5/r for i in range(j+1,n)])
        w=np.array(np.concatenate((w1,np.squeeze(wj))))
    
    P=np.identity(n)-2*np.outer(w,w)
    matrix_1=np.around(np.matmul(P,matrix),14)
    return alpha,r ,w, matrix_1
    

Bidiagonalize matrix using Householder Transforms

In [306]:
def bidi(a):
    m,n=np.shape(a)
    if m==n:
        for i in range(1,n):
            if i==1:
                _,_,_,math=hht(a,i,i)
                _,_,_,math=hht(math.T,i+1,i)
                print(math)
                print(i)
            elif i==n-1:
                _,_,_,math=hht(math.T,i,i)
                print(math)
                print(i)
            else:
                _,_,_,math=hht(math.T,i,i)
                _,_,_,math=hht(math.T,i+1,i)
                print(math)
                print(i)
        return math
    elif m>n:
        for i in range(1,n-1):
            if i==1:
                _,_,_,math=hht(a,i,i)
                _,_,_,math=hht(math.T,i+1,i)
                print(np.around(math,3))
                print(i)
            else:
                _,_,_,math=hht(math.T,i,i)
                _,_,_,math=hht(math.T,i+1,i)
                print(math)
                print(i)
        math=math.T
        for i in range(n-1,m):
            _,_,_,math=hht(math,i,i)
            print(np.around(math,3))
        return math[:n,:n]
    elif m<n:
        print("Try transposing the matrix argument")
        return 0
        

In [307]:
big=np.array([[4, 1 , -2, 2,6],
[1 ,2 ,0 ,1,9],
[-2 ,0, 3, -2,5],
[2 ,1 ,-2 ,-1,4],[1,8,-9,5,-2],[7,-6,1,4,3]])


In [308]:
aa=bidi(big)
mul=np.matmul(aa,aa.T)
mat=qrdecomp(mul)
print(np.around(mat,1))
print(mul)
print(np.diag(np.sqrt(mat)))
print(np.linalg.svd(big)[1])

[[ -8.66    0.      0.      0.      0.      0.   ]
 [ -8.573   4.849   3.584   0.08    0.134  -1.511]
 [ -0.      0.877   3.097  -1.566 -10.26    4.057]
 [ -0.     -0.74   -2.593  -1.576   7.963  -1.399]
 [ -0.      6.728   4.915   2.757   1.072  -5.197]]
1
[[-8.66025404  0.          0.          0.          0.          0.        ]
 [-8.57282528 -6.21758279  0.          0.          0.          0.        ]
 [ 0.          9.61322188 -0.31258394 -2.63370583  1.78443209  2.0847997 ]
 [ 0.          0.         -1.44535623 -1.40596125  6.23377201 -1.38699466]
 [ 0.         -0.         -2.66740399  1.72835392 11.33062039 -4.9532751 ]]
2
[[-8.66025404  0.          0.          0.          0.          0.        ]
 [-8.57282528 -6.21758279  0.          0.          0.          0.        ]
 [ 0.          9.61322188  3.81637299  0.          0.          0.        ]
 [ 0.          0.         -3.62653055 -3.41002096 -7.99302204  6.53601717]
 [ 0.          0.          0.          3.28554146  6.60688503 -4

C:\Users\pi\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sqrt
  


In [309]:
mat=qrdecomp(mul)

[[180.  38.   0.  -0.   0.]
 [ 38. 113.  14.   0.   0.]
 [  0.  14. 196. -32.  -0.]
 [  0.   0. -32.  12.  -0.]
 [  0.   0.  -0.  -0.   0.]]
[[193.  21.  -0.  -0.   0.]
 [ 21. 106.  26.   0.   0.]
 [  0.  26. 196.  -1.  -0.]
 [  0.   0.  -1.   7.  -0.]
 [  0.   0.  -0.  -0.   0.]]
[[196.  12.  -0.   0.   0.]
 [ 12. 120.  45.   0.   0.]
 [  0.  45. 178.  -0.  -0.]
 [  0.   0.  -0.   7.  -0.]
 [  0.   0.  -0.  -0.   0.]]
[[197.   8.  -0.   0.   0.]
 [  8. 157.  53.   0.  -0.]
 [  0.  53. 141.  -0.  -0.]
 [  0.   0.  -0.   7.  -0.]
 [  0.   0.  -0.  -0.   0.]]
[[198.   6.  -0.   0.   0.]
 [  6. 187.  38.   0.  -0.]
 [  0.  38. 110.  -0.  -0.]
 [  0.   0.  -0.   7.  -0.]
 [  0.   0.  -0.  -0.   0.]]
[[198.   6.  -0.   0.   0.]
 [  6. 198.  20.   0.  -0.]
 [  0.  20.  98.  -0.  -0.]
 [  0.   0.  -0.   7.  -0.]
 [  0.   0.  -0.  -0.   0.]]
[[199.   6.  -0.   0.   0.]
 [  6. 201.  10.   0.  -0.]
 [  0.  10.  95.  -0.  -0.]
 [  0.   0.  -0.   7.  -0.]
 [  0.  -0.  -0.  -0.   0.]]
[[199.   6.  

def givenr(n,j,cos,sin):
    a=np.identity(n)
    a[j-1,j-1]=cos
    a[j-1,j]=sin
    a[j,j-1]=-sin
    a[j,j]=cos
    return a
def givenl(n,j,cos,sin):
    a=np.identity(n)
    a[j-1,j-1]=cos
    a[j-1,j]=-sin
    a[j,j-1]=sin
    a[j,j]=cos
    return a    
givenr(10,2,0.1,0.5)


In [216]:

alpha=aa[3,3]
beta=aa[2,3]
givens=givenl(4,3,*cs(alpha,beta))
print(givens)

[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          0.94863141 -0.31638339]
 [ 0.          0.          0.31638339  0.94863141]]


In [217]:
aa=bidi(big)
aa=np.matmul(aa,aa.T)

[[ -8.66    0.      0.      0.      0.      0.   ]
 [ -8.573   4.849   3.584   0.08    0.134  -1.511]
 [ -0.      0.877   3.097  -1.566 -10.26    4.057]
 [ -0.     -0.74   -2.593  -1.576   7.963  -1.399]
 [ -0.      6.728   4.915   2.757   1.072  -5.197]]
1
[[-8.66025404  0.          0.          0.          0.          0.        ]
 [-8.57282528 -6.21758279  0.          0.          0.          0.        ]
 [ 0.          9.61322188 -0.31258394 -2.63370583  1.78443209  2.0847997 ]
 [ 0.          0.         -1.44535623 -1.40596125  6.23377201 -1.38699466]
 [ 0.         -0.         -2.66740399  1.72835392 11.33062039 -4.9532751 ]]
2
[[-8.66025404  0.          0.          0.          0.          0.        ]
 [-8.57282528 -6.21758279  0.          0.          0.          0.        ]
 [ 0.          9.61322188  3.81637299  0.          0.          0.        ]
 [ 0.          0.         -3.62653055 -3.41002096 -7.99302204  6.53601717]
 [ 0.          0.          0.          3.28554146  6.60688503 -4

In [218]:
ans=(np.around(aa,3))

In [219]:
np.sqrt(ans[0])

array([12.18577039,  7.30082187,  0.        ,  0.        ,  0.        ])

In [220]:
np.linalg.svd(big)

(array([[-0.16245186,  0.53339794, -0.02798506, -0.18185203, -0.7786487 ,
          0.221244  ],
        [-0.28915889,  0.5206853 ,  0.41464956,  0.3598046 ,  0.15758536,
         -0.56484264],
        [-0.30794088,  0.03148567,  0.48043499,  0.23508145,  0.2274779 ,
          0.75256482],
        [-0.08527243,  0.30441083,  0.13321856, -0.84075824,  0.41884342,
          0.00335218],
        [ 0.7674657 ,  0.52010942, -0.08163377,  0.20441433,  0.21280054,
          0.21621573],
        [-0.44597529,  0.28320019, -0.75634221,  0.18315994,  0.3105044 ,
          0.13743945]]),
 array([14.36941284, 13.92590074,  9.72817651,  2.61305043,  0.35186991]),
 array([[-0.19819813,  0.55600986, -0.54153504,  0.14896356, -0.57975873],
        [ 0.40949869,  0.31171079, -0.42933992,  0.35570028,  0.65137681],
        [-0.59289146,  0.49541821,  0.1242987 , -0.4285435 ,  0.45159807],
        [-0.39522735,  0.08930328,  0.4186279 ,  0.81185089,  0.03832861],
        [-0.53413798, -0.58333107, -0.575